
# G2Net Playground GPU



This notebook has been created to serve as a guided tour to training G2Net models by using Colab GPU. It **MAKES USE** of the code in the project-like tree uploaded to my GitHub as well, a code that rocketed the team (ROC & Roll) to top 8% and a bronze medal in its first competition. With all that said, let's ROC it! (Hope you get the joke)

## 1. Introduction

### 1.1 Problem Description

Not only a network of Gravitational Waves, Geophysics and Machine Learning experts, G2Net was also released as a Kaggle competition. I'm pretty sure that you have heard about the discovery of Gravitational Waves (GW), signals from colliding binary black holes, back in 2015. If not, you can refresh your memory with [The Sound of Two Black Holes Colliding](https://www.youtube.com/watch?v=QyDcTbR-kEA). 

The aim of this competition is to detect GW signals from the mergers of binary black holes. Specifically, the participant was expected to build a model to analyse synthetic GW time-series data from a network of Earth-based detectors (LIGO Hanford, LIGO Livingston and Virgo). The [Data](https://www.kaggle.com/c/g2net-gravitational-wave-detection/data) was simulated with a sampling rate of 2048 Hz. Each of the time-series originated by the corresponding detector comprises a channel (three in total).

In the context of this notebook, data has been already standardised (with training set mean and standard deviation), transposed (to ease channels last format) and saved to TensorFlow Records format. Such data can be found in my Kaggle profile, [Training Dataset](https://www.kaggle.com/salbeal94/g2net-float32-train) and [Test Dataset](https://www.kaggle.com/salbeal94/g2net-float32-test). Since it is a classification task, the output is the black hole merger occurence probability. As per the evaluation/validation metric, it is the AUC score.

### 1.2 Implemented Model

After many iterations, the model used for the competition ended up being a 2D Convolutional Neural Network (CNN) preceded by a series of time-series processing techniques. Even the preprocessing has been here implemented as part of the model to stay loyal to the end-to-end philosophy. Such a model contains a series of building blocks several of them presented as trainable Tensorflow Keras layers. These are described as follows:
- **Tukey Window (trainable/non-trainable)**: Introduces a tappering effect that forces the signal amplitude to decay until having zero values at the ends. It is applied to avoid artefacts stemming from discontinuities when taking Fourier transforms or similar.

- **Bandpass Filter (trainable/non-trainable)**: Applies a filter with the frequency response of a Butterworth filter. The idea is to filter out or attenuate frequencies that have nothing to do with the merger.

- **Constant-Q Transform (trainable/non-trainable)**: Transforms the time-domain signal into the time-frequency domain. In other words, it converts a time-domain signal to a spectrogram. Particularly, the PyTorch CQT1992v2 implementation from [nnAudio](https://github.com/KinWaiCheuk/nnAudio) has been taken as reference. It has been re-implemented in TensorFlow for being one of the most cost-effective solutions offered by the library. As an additional functionality, the layer keeps track of maximum spectrogram magnitudes and normalises its values to a preset range for the sake of stability. The output spectrogram is later resized with bilinear interpolation to adapt it to the downstream CNN recommended input sizes.

- **Channel Permutation (non-trainable)**: Randomly decides whether to apply a stochastic permutation of the channels. The aim is to make the prediction a bit less independent of the detector it comes from, pressumably acting as a regularisation layer.

- **Spectral Masking (non-trainable)**: Randomly decides whether to stochastically mask certain time or frequency bands. Similar to the permutation, the idea is for this layer to act as a regularising operation.

- **Convolutional Neural Network (trainable/non-trainable)**: Used as backbone to extract the relevant features to be ingested by a single fully-connected neuron with sigmoid activation (after flattening). Given its performance-complexity trade-off in the ImageNet dataset, the EfficientNet family from [AutoML](https://github.com/google/automl) was selected as a more than appropriate model for this purpose.

![G2Net Model](https://github.com/salvaba94/G2Net/blob/main/img/Model.png?raw=true "G2Net Model")

## 2. Configuration

### 2.1 Environment Configuration

Before starting with implementation-specific details, let's configure some aspects of the environment: 

- Make sure the Colab environment type is set to GPU going to ```Runtime → Change runtime type → GPU``` and check its characteristics.
- Mount your Google Drive to save any output model after the execution.
- Install any library that might be missing from the defaults.



In [1]:
gpu_info = !nvidia-smi
gpu_info = "\n".join(gpu_info)
if gpu_info.find("failed") >= 0:
  print("GPU execution environment is not activated.")
else:
  print(gpu_info)

Sun Nov 20 18:15:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.02    Driver Version: 526.98       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:0A:00.0  On |                  Off |
|  0%   23C    P8    19W / 508W |   1631MiB / 24564MiB |     13%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# from google.colab import drive
# drive.mount("/content/gdrive", force_remount = True)


In [3]:
# !pip install tensorflow-addons
# !pip install adabelief-tf
# !git clone https://github.com/salvaba94/G2Net.git g2net
# !git clone https://github.com/google/automl.git g2net/src/automl

/bin/sh: 1: pip: not found
/bin/sh: 1: pip: not found
fatal: destination path 'g2net/src/automl' already exists and is not an empty directory.


### 2.2 Code Configuration 


Now it is high time to start with the Python specifics. Run the following cell to import the necessary libraries for the code to work.

In [4]:
import sys
import os
import warnings
import copy
import pandas as pd
import numpy as np
import multiprocessing as mp
import tensorflow as tf
import seaborn as sns
# from tensorflow import AUTOTUNE
from tensorflow.python.client import device_lib
# from tensorflow.keras.layers import Input, Dense, Flatten
# from tensorflow.keras.layers.preprocessing import Resizing
from adabelief_tf import AdaBeliefOptimizer
from datetime import datetime
from pathlib import Path, os
from functools import partial
from scipy import signal
from scipy import interpolate
from typing import Tuple, Union, Mapping
import matplotlib.pyplot as plt

%matplotlib inline

2022-11-20 18:15:46.861633: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-20 18:15:47.496749: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-20 18:15:47.496835: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-11-20 18:15:47.496843: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

Set the configuration variables necessary for the code to run. Here is where you should come if you would like to play with the model. Don't worry if some of the variables are not clear enough at this point. Their utility can be derived as these are used all along the code.


In [5]:
class Config:
### General data #############################################################
    N_SAMPLES, N_DETECT = 4096, 3

### Training #################################################################
    FROM_TFR = False
    MODEL_TRAIN = True
    MODEL_SAVE_NAME = "Model_Ref.h5"
    MODEL_PRELOAD = False
    MODEL_PRELOAD_NAME = "Model_Ref.h5"
    HISTORY_NAME = "history_train.csv"

    MODEL_PATH = Path("models")
    CKPT_PATH = Path("checkpoints")

    SPLIT = 0.98
    SEED_SPLIT = 21
    BATCH_SIZE = 64
    BATCH_SIZE_TEST = 32
    EPOCHS = 1
    LEARNING_RATE = 0.0001
    
### Prediction ################################################################
    MODEL_PREDICT = True
    PREDICTIONS_NAME = "submission.csv"


### Model ####################################################################
    TUKEY_SHAPE = 0.25
    TRAINABLE_TUKEY = False

    DEGREE_FILT = 6
    F_BAND_FILT = (20., 500.)
    TRAINABLE_FILT = True

    SAMPLE_RATE = 2048
    F_BAND_SPEC = (20., 500.)
    HOP_LENGTH = 64
    BINS_PER_OCTAVE = 12
    WINDOW_CQT = "hann"
    TRAINABLE_CQT = False
    
    IMAGE_SIZE = 120

    P_PERM = 0.1

    P_MASK = 0.15
    N_MAX_MASK = 2
    W_MASK = (IMAGE_SIZE // 64, IMAGE_SIZE // 6)
    
    MODEL_ID = "efficientnetv2-b0"
    MODEL_ID_WEIGHTS = "imagenet"

### Plotting #################################################################
    PLOT_EXAMPLE = True
    PLOT_TEST = False

When making use of Colab GPUs, data ingested by the model should be stored locally. Taking into consideration the great size of the datasets, data can be affordably downloaded in several ways:
 * Firstly, one can use [Kaggle API](https://colab.research.google.com/github/corrieann/kaggle/blob/master/kaggle_api_in_colab.ipynb#scrollTo=OppyMnCuWjzJ).
 * Secondly, you can simply upload the data to your Google Drive (assuming you have enough space) and manually copy it from the folder is has been mounted into. 
 * Thirdly, one can use Google Cloud Storage (GCS) buckets. Fortunately, Kaggle is so integrated with Google that public datasets get uploaded to Google Cloud. 
 
We'll use the third method due to its speed when it comes to downloading files. To know the public GCS bucket addresses one should just run the following in a Kaggle notebook:
```
from kaggle_datasets import KaggleDatasets
MODEL_BUCK = KaggleDatasets().get_gcs_path("automl-efficientnetv2-ckpt")
TEST_PATH = KaggleDatasets().get_gcs_path("g2net-float32-test")
TRAIN_PATH = KaggleDatasets().get_gcs_path("g2net-float32-train")
```

The output addresses should be pasted below as raw strings (check them from time to time as they are periodically changed).

In [6]:
project_directory = r'/mnt/c/Users/Chris/OneDrive\ -\ Georgia\ Institute\ of\ Technology/GaTech/CS7643/Deep\ Learning\Group Project/G2Net'
os.environ["MODEL_BUCK"] = project_directory + r'/tfrecords/effnet'
os.environ["TEST_BUCK"] = project_directory + r'/tfrecords/test'
os.environ["TRAIN_BUCK"] = project_directory + r'/tfrecords/train'
MODEL_BUCK = os.environ["MODEL_BUCK"]
TEST_BUCK = os.environ["TEST_BUCK"]
TRAIN_BUCK = os.environ["TRAIN_BUCK"]

OV_PATH = Path(project_directory)
SRC_PATH = str(Path(OV_PATH, "src"))  # SRC_PATH = str(Path(OV_PATH, "g2net", "src"))

CKT_PATH = MODEL_BUCK + "/" + Config.MODEL_ID_WEIGHTS
TRAIN_PATH = TRAIN_BUCK + "/train"
TEST_PATH = TEST_BUCK + "/test"

AUTOML_PATH = str(Path(SRC_PATH, "automl"))
EFFNETV2_PATH = str(Path(AUTOML_PATH, "efficientnetv2"))

print('OV_PATH:       ', OV_PATH)
print('SRC_PATH:      ', SRC_PATH)
print('CKT_PATH:      ', CKT_PATH)
print('TRAIN_PATH:    ', TRAIN_PATH)
print('TEST_PATH:     ', TEST_PATH)
print('AUTOML_PATH:   ', AUTOML_PATH)
print('EFFNETV2_PATH: ', EFFNETV2_PATH)

# if SRC_PATH not in sys.path:
#     sys.path.append(SRC_PATH)
#     sys.path.append(AUTOML_PATH)
#     sys.path.append(EFFNETV2_PATH)
#
# from utilities import PlottingUtilities, GeneralUtilities
# from ingest import TFRDatasetCreator, NPYDatasetCreator, DatasetGeneratorTF
# from models import G2NetEfficientNet
# from train import RocLoss, Acceleration, CosineAnnealingRestarts


OV_PATH:        /mnt/c/Users/Chris/OneDrive\ -\ Georgia\ Institute\ of\ Technology/GaTech/CS7643/Deep\ Learning\Group Project/G2Net
SRC_PATH:       /mnt/c/Users/Chris/OneDrive\ -\ Georgia\ Institute\ of\ Technology/GaTech/CS7643/Deep\ Learning\Group Project/G2Net/src
CKT_PATH:       /mnt/c/Users/Chris/OneDrive\ -\ Georgia\ Institute\ of\ Technology/GaTech/CS7643/Deep\ Learning\Group Project/G2Net/tfrecords/effnet/imagenet
TRAIN_PATH:     /mnt/c/Users/Chris/OneDrive\ -\ Georgia\ Institute\ of\ Technology/GaTech/CS7643/Deep\ Learning\Group Project/G2Net/tfrecords/train/train
TEST_PATH:      /mnt/c/Users/Chris/OneDrive\ -\ Georgia\ Institute\ of\ Technology/GaTech/CS7643/Deep\ Learning\Group Project/G2Net/tfrecords/test/test
AUTOML_PATH:    /mnt/c/Users/Chris/OneDrive\ -\ Georgia\ Institute\ of\ Technology/GaTech/CS7643/Deep\ Learning\Group Project/G2Net/src/automl
EFFNETV2_PATH:  /mnt/c/Users/Chris/OneDrive\ -\ Georgia\ Institute\ of\ Technology/GaTech/CS7643/Deep\ Learning\Group Project

Run the next cell to tune the paths and add G2Net source code and AutoML EfficientNet model paths to Python path.

In [7]:
OV_PATH = Path("/content")
SRC_PATH = str(Path(OV_PATH, "g2net", "src"))
CKT_PATH = MODEL_BUCK + "/" + Config.MODEL_ID_WEIGHTS
TRAIN_PATH = TRAIN_BUCK + "/train"
TEST_PATH = TEST_BUCK + "/test"

AUTOML_PATH = str(Path(SRC_PATH, "automl"))
EFFNETV2_PATH = str(Path(AUTOML_PATH, "efficientnetv2"))

if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)
    sys.path.append(AUTOML_PATH)
    sys.path.append(EFFNETV2_PATH)
    
from utilities import PlottingUtilities, GeneralUtilities
from ingest import TFRDatasetCreator, NPYDatasetCreator, DatasetGeneratorTF
from models import G2NetEfficientNet
from train import RocLoss, Acceleration, CosineAnnealingRestarts


ImportError: libffi.so.7: cannot open shared object file: No such file or directory

The following cell automatically configures the TensorFlow strategy and device (not really relevant for GPU) and sets the data type to use.

In [8]:
# Create strategy and define data types for data and tensorflow models
strategy, device = Acceleration.get_acceleration()
dtype = tf.float32 # Do not modify since TF Records are in tf.float32

NameError: name 'Acceleration' is not defined

Since each TF Record contains several examples (each with ID, time-domain signals and label if any), one might lose track of the number of examples in training and test sets. Actually, it needs to be known to figure out the number of steps per epoch. Such information is contained in two CSV that need to be locally stored. This is the purpose of the following lines.

In [9]:
!mkdir -p g2net-float32-train
!mkdir -p g2net-float32-test
!gsutil -m cp -r $TRAIN_BUCK/* g2net-float32-train
!gsutil -m cp -r $TEST_BUCK/* g2net-float32-test

/bin/sh: 1: gsutil: not found
/bin/sh: 1: gsutil: not found


## 3. Source Code

### 3.1 Model Training

At this point, it only remains to prepare the datasets with the classes defined above before proceding with training. Of note is that a small proportion of the training dataset is stripped and used for validation purposes.

In [10]:
# Prepare original dataframes 
sub_file = OV_PATH.joinpath("g2net-float32-test", "sample_submission.csv")
train_labels_file = OV_PATH.joinpath("g2net-float32-train", "training_labels.csv")

train_df_ori = pd.read_csv(train_labels_file)
test_df_ori = pd.read_csv(sub_file)

train_df = pd.DataFrame([Path(x).stem for x in tf.io.gfile.glob(TRAIN_PATH + "/*.tfrec")], columns = ["id"])
train_df = train_df.sample(frac = 1, random_state = Config.SEED_SPLIT).reset_index(drop = True)
test_df = pd.DataFrame([Path(x).stem for x in tf.io.gfile.glob(TEST_PATH + "/*.tfrec")], columns = ["id"])

n_split = np.int32(train_df.shape[0] * Config.SPLIT)
training_df = train_df.loc[:n_split - 1, :]
validation_df = train_df.loc[n_split:, :]
        
training_gen = DatasetGeneratorTF(training_df, TRAIN_PATH, batch_size = Config.BATCH_SIZE, dtype = dtype)
validation_gen = DatasetGeneratorTF(validation_df, TRAIN_PATH, batch_size = Config.BATCH_SIZE, dtype = dtype)
test_gen = DatasetGeneratorTF(test_df, TEST_PATH, batch_size = Config.BATCH_SIZE_TEST, dtype = dtype)
    
training_ds = training_gen.get_dataset(buffer_size = 2048)
validation_ds = validation_gen.get_dataset(buffer_size = 2048)
test_ds = test_gen.get_dataset(shuffle = False, repeat = False, target = False)

# Estimate number of steps per train, validation and test sets
ns_training = np.int32(train_df_ori.shape[0] * Config.SPLIT)
ns_validation = train_df_ori.shape[0] - ns_training
ns_test = test_df_ori.shape[0]
spe_training = np.int32(np.ceil(ns_training / Config.BATCH_SIZE))
spe_validation = np.int32(np.ceil(ns_validation / Config.BATCH_SIZE))
spe_test = np.int32(np.ceil(ns_test / Config.BATCH_SIZE_TEST))

test_ds_id = test_ds.map(lambda data, identity: tf.strings.unicode_encode(
    identity, "UTF-8"))
test_ds_id = test_ds_id.unbatch()
test_ids = next(iter(test_ds_id.batch(test_df_ori.shape[0]))).numpy().astype("U")
test_ds = test_ds.map(lambda data, identity: data)

FileNotFoundError: [Errno 2] No such file or directory: '/content/g2net-float32-train/training_labels.csv'

The building blocks of the model are already implemented in the cloned G2Net source code. The same applies to the model, which, in this case, is compiled outside the scope of the method that builds it. Now we are ready to build the model, preload model weights from local storage (weights other than those provided by AutoML) and train it.

In [11]:
# Create model, compile and display summary within the scope of the 
# distribution strategy
tf.keras.backend.clear_session()
with strategy.scope():
        model_gen = G2NetEfficientNet(input_shape = 
                                      (Config.N_SAMPLES, Config.N_DETECT),
                                      window_shape = Config.TUKEY_SHAPE,
                                      trainable_window = Config.TRAINABLE_TUKEY,
                                      sample_rate = Config.SAMPLE_RATE,
                                      degree_filt = Config.DEGREE_FILT,
                                      f_band_filt = Config.F_BAND_FILT,
                                      trainable_filt = Config.TRAINABLE_FILT,
                                      hop_length = Config.HOP_LENGTH,
                                      f_band_spec = Config.F_BAND_SPEC,
                                      bins_per_octave = Config.BINS_PER_OCTAVE,
                                      window_cqt = Config.WINDOW_CQT,
                                      resize_shape = 
                                      (Config.IMAGE_SIZE, Config.IMAGE_SIZE),
                                      p_perm = Config.P_PERM,
                                      p_mask = Config.P_MASK, 
                                      n_max_mask_t = Config.N_MAX_MASK,
                                      w_mask_t = Config.W_MASK,
                                      n_max_mask_f = Config.N_MAX_MASK,
                                      w_mask_f = Config.W_MASK,
                                      strategy = device)
        model = model_gen.get_model(effnet_id = Config.MODEL_ID, 
                                    weights = Config.MODEL_ID_WEIGHTS)
        optimizer = tf.keras.optimizers.Adam(learning_rate = Config.LEARNING_RATE)
        # optimizer = AdaBeliefOptimizer(learning_rate = Config.LEARNING_RATE, 
        #                                amsgrad = False, print_change_log = False)
        model.compile(optimizer = optimizer, 
                  loss = "binary_crossentropy",
                  metrics = [tf.keras.metrics.AUC()])

model.summary()

NameError: name 'strategy' is not defined

In [ ]:
if Config.MODEL_PRELOAD:
    pretrained_model = Config.MODEL_PATH.joinpath(Config.MODEL_PRELOAD_NAME)
    local = tf.train.CheckpointOptions(experimental_io_device = "/job:localhost")
    if tf.io.gfile.isdir(pretrained_model):
        pretrained_model = tf.train.latest_checkpoint(pretrained_model)
    model.load_weights(pretrained_model, options = local)

In [ ]:
# Train model with training and validation sets with checkpoints and control 
# over training validation loss plateaus
if Config.MODEL_TRAIN:
    lr_callback = tf.keras.callbacks.ReduceLROnPlateau(patience = 1, monitor = "val_loss", cooldown = 0, verbose = 1)
    
    local = tf.train.CheckpointOptions(experimental_io_device='/job:localhost')
    check_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath = Config.CKPT_PATH.joinpath("ckpt-{epoch:d}"),
            save_weights_only = True,
            monitor = "val_auc",
            mode = "max",
            save_best_only = False,
            options = local)
    
    train_history = model.fit(training_ds, epochs = Config.EPOCHS, batch_size = Config.BATCH_SIZE, 
                              validation_data = validation_ds, steps_per_epoch = spe_training,
                              validation_steps = spe_validation, callbacks = [lr_callback, check_callback])

    #best_model_path = tf.train.latest_checkpoint(Config.CKPT_PATH)
    #model.load_weights(best_model_path, options = local)

    Config.MODEL_PATH.mkdir(parents = True, exist_ok = True)
    model.save_weights(Config.MODEL_PATH.joinpath(Config.MODEL_SAVE_NAME), options = local)

    train_hist_df = pd.DataFrame(train_history.history)
    train_hist_df.to_csv(Config.HISTORY_NAME, index = False)

### 3.2 Predicting with the Model

Once you are happy with the model, it is time to make predictions on the test set and explore a bit what is actually doing. First, try to obtain the predictions for the full test dataset and plot the output distribution.

In [ ]:
# Predict on test set and save to submission file
if Config.MODEL_PREDICT:
    preds_test = model.predict(test_ds, batch_size = Config.BATCH_SIZE_TEST, 
                               steps = spe_test, verbose = 1)

    sub_df = pd.DataFrame({
        "id": test_ids,
        "target": preds_test.flatten()
    })

    sub_df = sub_df.sort_values("id").reset_index(drop = True)
        
    sub_df.to_csv(Config.PREDICTIONS_NAME, index = False)

    plt.style.use("seaborn")
    print()
    print("Test dataset output distribution")
    sns.displot(sub_df, x = "target", kind = "kde")

Now let's try to break down the model into its constituent parts and plot the intermediate data for a single example. You might see that, depending on the example, visually identifying a merger chirp becomes difficult or even impossible mainly due other low and high frequency sources of noise (in this case simulated).

In [ ]:
if Config.PLOT_EXAMPLE:

    permute = model.get_layer("permute")
    window = model.get_layer("window")
    bandpass = model.get_layer("bandpass")
    cqt = model.get_layer("cqt")
    resize = model.get_layer("resize")
    permute = model.get_layer("permute")
    mask = model.get_layer("mask")
    effnet = model.get_layer(Config.MODEL_ID)
    flatten = model.get_layer("flatten")
    dense = model.get_layer("dense")
        
    if Config.PLOT_TEST:
        for data, _ in test_ds.take(1):
            x_ref = data.numpy()[0, ...][np.newaxis, ...]
    else:
        for data, _ in training_ds.take(1):
            x_ref = data.numpy()[0, ...][np.newaxis, ...]

    x = np.squeeze(x_ref)
    y = x_ref
        
    y = window(y)
    y_win = np.squeeze(y.numpy())
    y = bandpass(y)
    y_band = np.squeeze(y.numpy())
    y = cqt(y, training = False)
    y = resize(y)
    y_spec = np.squeeze(y.numpy())
    y = permute(y, training = True)
    y = mask(y, training = True)
    y_masked = np.squeeze(y.numpy())
    y = effnet(y)
    y = flatten(y)
    y = dense(y)
    y_dense = np.squeeze(y.numpy())

In [ ]:
if Config.PLOT_EXAMPLE:
    print("Standardised signals")
    PlottingUtilities.plot_wave(x)

In [ ]:
if Config.PLOT_EXAMPLE:
    print("Tappered signals")
    PlottingUtilities.plot_wave(y_win)

In [ ]:
if Config.PLOT_EXAMPLE:
    print("Filtered signals")
    PlottingUtilities.plot_wave(y_band)

In [ ]:
if Config.PLOT_EXAMPLE:
    print("CQT spectrogram")
    PlottingUtilities.plot_spectrogram(y_spec)

In [ ]:
if Config.PLOT_EXAMPLE:
    print("Masked CQT spectrogram")
    PlottingUtilities.plot_spectrogram(y_masked)

Don't forget to save the results to your Google Drive! Make sure you have enough space as some models are truly heavy.

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount = True)
os.environ["OUT_NAME"] = "g2net_output"
!echo $OUT_NAME
!mkdir -p $OUT_NAME
!cp -r checkpoints $OUT_NAME
!cp -r models $OUT_NAME
!cp -r *.csv $OUT_NAME
!cp -r $OUT_NAME /content/gdrive/MyDrive/
drive.flush_and_unmount()


The best G2Net single model was obtained with trainable Tukey window and bandpass layers, a non-trainable CQT, a resize size of 384, an EfficientNet v2 S backbone and training in a Tesla V100 (16GB) GPU with a batch size of 32 (learning rates in the range 0.0003 to 0.00001. The score was boosted with an averaging ensemble of several models with different settings. 

The ROC & Roll team encourages you to play with the code, propose improvements and even introduce new layers that might be of use!